<a href="https://colab.research.google.com/github/aim56009/Bias_GAN/blob/master/code/run_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports 

In [1]:
from google.colab import drive
drive.mount("/content/gdrive")
#%cd "/content/gdrive/MyDrive/data_gan"

In [3]:
!git clone https://github.com/aim56009/Bias_GAN.git

Cloning into 'Bias_GAN'...
remote: Enumerating objects: 192, done.
remote: Counting objects: 100% (93/93), done.
remote: Compressing objects: 100% (83/83), done.
remote: Total 192 (delta 43), reused 23 (delta 7), pack-reused 99
Receiving objects: 100% (192/192), 79.63 MiB | 31.02 MiB/s, done.
Resolving deltas: 100% (75/75), done.


In [4]:
%%capture
!pip install pytorch_lightning

In [5]:
!pip install importlib-metadata==4.0.1
!pip install xarray==0.18.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Attempting uninstall: importlib-metadata
    Found existing installation: importlib-metadata 5.2.0
    Uninstalling importlib-metadata-5.2.0:
      Successfully uninstalled importlib-metadata-5.2.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
markdown 3.4.1 requires importlib-metadata>=4.4; python_version < "3.10", but you have importlib-metadata 4.0.1 which is incompatible.
gym 0.25.2 requires importlib-metadata>=4.8.0; python_version < "3.10", but you have importlib-metadata 4.0.1 which is incompatible.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 807.3/807.3 KB 14.1 MB/s eta 0:00:00
  Attempting uninstall: xarray
    Found existing installation: xarray 2022.12

In [6]:
from argparse import ArgumentParser
import warnings
warnings.filterwarnings('ignore')
from dataclasses import dataclass, field
from typing import List
import getpass

from Bias_GAN.code.src.trainer import train_cycle_gan

# Config

In [16]:
@dataclass
class Config:
    """ 
    Training configuration parameters. For model evaluation parameters see
    src/configuration.py.
    """
    
    scratch_path: str = '/results'
    tensorboard_path: str = f'{scratch_path}/'
    checkpoint_path: str = f'{scratch_path}/'
    #checkpoint_path: str = '/data/checkpoint_folder/last.ckpt'
    config_path: str = f'{scratch_path}/'
    poem_path: str = f"/content/gdrive/MyDrive/data_gan/pr_gfdl-esm4_historical_regionbox_1979-2014.nc"
    era5_path: str = f"/content/gdrive/MyDrive/data_gan/pr_W5E5v2.0_regionbox_era5_1979-2014.nc"
   

    results_path: str = f'{scratch_path}/'
    projection_path: str = None

    train_start: int = 1979
    train_end: int = 2000 # set to 2000 for full run
    valid_start: int = 2001
    valid_end: int = 2004
    test_start: int = 2004
    test_end: int = 2014
    
    model_name: str = 'tibet_gan'

    epochs: int = 10 # set to 250 for reproduction
    progress_bar_refresh_rate: int = 1
    train_batch_size: int = 1
    test_batch_size: int = 64
    transforms: List = field(default_factory=lambda: ['log', 'normalize_minus1_to_plus1'])
    rescale: bool = False
    epsilon: float = 0.0001
    lazy: bool = False
    log_every_n_steps: int = 2 ### was 10
    norm_output: bool = True
    running_bias: bool = False


def main():
    _ = train_cycle_gan(Config())

# Training

In [17]:
import pytorch_lightning as pl
from pytorch_lightning.loggers import TensorBoardLogger

from Bias_GAN.code.src.model import CycleGAN
from Bias_GAN.code.src.data import DataModule
from Bias_GAN.code.src.utils import get_version, set_environment, get_checkpoint_path, save_config
from Bias_GAN.code.src.callbacks import get_cycle_gan_callbacks

import json


config = Config()
version = get_version()
print(f'Running model: {version}')
print(json.dumps(config.__dict__, indent=4))
checkpoint_path = get_checkpoint_path(config, version)


tb_logger = TensorBoardLogger(config.tensorboard_path,
                           name=config.model_name,
                           default_hp_metric=False,
                           version = version)

trainer = pl.Trainer(gpus = 1,
                         max_epochs = config.epochs,
                         precision = 16, 
                         callbacks = get_cycle_gan_callbacks(checkpoint_path),
                         num_sanity_val_steps = 1,
                         logger = tb_logger,
                         log_every_n_steps = config.log_every_n_steps,
                         deterministic = False)


INFO:pytorch_lightning.utilities.rank_zero:Using 16bit native Automatic Mixed Precision (AMP)
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


Running model: 2023_01_04/12h_11m_18s_ID_e4f4af0a-8c28-11ed-8996-0242ac1c000c
{
    "scratch_path": "/results",
    "tensorboard_path": "/results/",
    "checkpoint_path": "/results/",
    "config_path": "/results/",
    "poem_path": "/content/gdrive/MyDrive/data_gan/pr_gfdl-esm4_historical_regionbox_1979-2014.nc",
    "era5_path": "/content/gdrive/MyDrive/data_gan/pr_W5E5v2.0_regionbox_era5_1979-2014.nc",
    "results_path": "/results/",
    "projection_path": null,
    "train_start": 1979,
    "train_end": 2000,
    "valid_start": 2001,
    "valid_end": 2004,
    "test_start": 2004,
    "test_end": 2014,
    "model_name": "tibet_gan",
    "epochs": 10,
    "progress_bar_refresh_rate": 1,
    "train_batch_size": 1,
    "test_batch_size": 64,
    "transforms": [
        "log",
        "normalize_minus1_to_plus1"
    ],
    "rescale": false,
    "epsilon": 0.0001,
    "lazy": false,
    "log_every_n_steps": 2,
    "norm_output": true,
    "running_bias": false
}


In [18]:
tb_logger = TensorBoardLogger(config.tensorboard_path,
                           name=config.model_name,
                           default_hp_metric=False,
                           version = version)

# Run

In [ ]:
main()

INFO:lightning_lite.utilities.seed:Global seed set to 42
INFO:pytorch_lightning.utilities.rank_zero:Using 16bit native Automatic Mixed Precision (AMP)
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


Running model: 2023_01_04/12h_11m_20s_ID_e60afbba-8c28-11ed-8996-0242ac1c000c
{
    "scratch_path": "/results",
    "tensorboard_path": "/results/",
    "checkpoint_path": "/results/",
    "config_path": "/results/",
    "poem_path": "/content/gdrive/MyDrive/data_gan/pr_gfdl-esm4_historical_regionbox_1979-2014.nc",
    "era5_path": "/content/gdrive/MyDrive/data_gan/pr_W5E5v2.0_regionbox_era5_1979-2014.nc",
    "results_path": "/results/",
    "projection_path": null,
    "train_start": 1979,
    "train_end": 2000,
    "valid_start": 2001,
    "valid_end": 2004,
    "test_start": 2004,
    "test_end": 2014,
    "model_name": "tibet_gan",
    "epochs": 10,
    "progress_bar_refresh_rate": 1,
    "train_batch_size": 1,
    "test_batch_size": 64,
    "transforms": [
        "log",
        "normalize_minus1_to_plus1"
    ],
    "rescale": false,
    "epsilon": 0.0001,
    "lazy": false,
    "log_every_n_steps": 2,
    "norm_output": true,
    "running_bias": false
}


INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type          | Params
----------------------------------------
0 | d_A   | Discriminator | 2.8 M 
1 | d_B   | Discriminator | 2.8 M 
2 | g_A2B | Generator     | 449 K 
3 | g_B2A | Generator     | 449 K 
----------------------------------------
6.4 M     Trainable params
0         Non-trainable params
6.4 M     Total params
12.849    Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]